In [1]:
import numpy as np
import pandas as pd
import lal
import lalsimulation as lalsim
import jax.numpy as jnp
import ripple
from astropy.cosmology import Planck18
%env XLA_PYTHON_CLIENT_ALLOCATOR=platform
import astropy.units as u
import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal
import lalsimulation as lalsim
import numpy as np
import os.path as op
import sys
import pandas as pd
import paths
import jax.numpy as jnp
from tqdm import tqdm, trange
import weighting
import scipy.integrate as sint
import intensity_models
import fisher_snrs
from ripplegw import waveforms as rip_wf
#from fisher_snrs import compute_snrs
import mock_injections
from mock_injections import *
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import jax
from importlib import reload 

/tmp/ipykernel_426064/2168093336.py:3: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  import lal


env: XLA_PYTHON_CLIENT_ALLOCATOR=platform


/mnt/home/amoran/miniforge3/envs/GW_all/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-10-13 10:04:05.192343: W external/xla/xla/service/gpu/nvptx_compiler.cc:836] The NVIDIA driver's CUDA version is 12.4 which is older than the PTX compiler version (12.9.86). Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [2]:
reload(fisher_snrs)

<module 'fisher_snrs' from '/mnt/home/amoran/GW_pop_cosmo_inference/src/fisher_snrs.py'>

In [8]:
ASD_FILES = {"CE": f"sensitivity_files/LIGO-P1600143-v18-CE-ASD.txt",
            "aplus": f"sensitivity_files/AplusDesign.txt",
            "aligo": f"sensitivity_files/aligo_O4high.txt",
            'o3_PSD': f"sensitivity_files/H1_o3_PSD.txt"}
population_parameters = dict()
config_file = '../reproduce/configs/config2.txt'
outfile = 'mock_injections_1det2.h5'

population_parameters = dict()

with open(config_file) as param_file:
    for line in param_file:
        (key, val) = line.split('=')
        population_parameters[key.strip()] = val.strip()
        try:
            population_parameters[key.strip()] = float(val.strip())
        except ValueError:
            pass
snr_threshold = population_parameters.pop('snr_threshold')
ndraw = int(population_parameters.pop('ndraw', int(2e4)))
sensitivity = population_parameters.pop('sensitivity', 'aligo')
detectors = population_parameters.pop('detectors', 'H1').split(',')

custom_cosmo = intensity_models.FlatwCDMCosmology(population_parameters['h'], population_parameters['Om'], population_parameters['w'], population_parameters['zmax'])
population_parameters['cosmo'] = custom_cosmo
print("Using the following custom population_parameters: " + str(population_parameters))

Using the following custom population_parameters: {'h': 0.674, 'Om': 0.315, 'w': -1.0, 'a': 1.8, 'b': -0.71, 'c': 2.9, 'mpisn': 31.0, 'mbhmax': 36.0, 'sigma': 0.1, 'fpl': 0.21, 'lam': 3.7, 'kappa': 7.0, 'zp': 3.0, 'mpisndot': 0.0, 'zmax': 1.9, 'mbh_min': 5.0, 'delta_m': 2.5, 'beta': -1.0, 'cosmo': FlatwCDMCosmology(h=0.674, Om=0.315, w=-1.0, zmax=1.9, ninterp=1024, zinterp=Array([0.00000000e+00, 1.04131475e-03, 2.08371384e-03, ...,
       1.89396980e+00, 1.89698333e+00, 1.90000000e+00],      dtype=float64), dcinterp=Array([0.00000000e+00, 4.63057878e-03, 9.26369560e-03, ...,
       5.15257946e+00, 5.15722323e+00, 5.16186519e+00],      dtype=float64), dlinterp=Array([0.00000000e+00, 4.63540067e-03, 9.28299849e-03, ...,
       1.49114093e+01, 1.49403897e+01, 1.49694090e+01],      dtype=float64), ddlinterp=Array([4.44795252, 4.45502339, 4.46209345, ..., 9.61530365,
       9.61819823, 9.62109004], dtype=float64), vcinterp=Array([0.00000000e+00, 4.15905286e-07, 3.32997856e-06, ...,
       5

In [4]:
def compare_all_snrs(test_df, sensitivity='aligo', fmin=20, fmax=1024, f_ref=20.0):
    """
    Compute and compare SNRs for each event using:
    (1) JAX IMRPhenomD (already done)
    (2) lal IMRPhenomD
    (3) Ripple Pv2
    (4) lalsim Pv2
    Returns: pandas.DataFrame with SNRs for all methods.
    """

    freqs_psd, asd = np.loadtxt(ASD_FILES[sensitivity], unpack=True)
    psd_interp = interp1d(freqs_psd, asd**2, bounds_error=False, fill_value="extrapolate")
    snrs_jax = fisher_snrs.compute_snrs_batch(test_df, detectors=['H1'], sensitivity=sensitivity,
                                  fmin=fmin, fmax=fmax, f_ref=f_ref)
    snrs_jax = np.array(snrs_jax) #what we had
    
    # lalsim phnomD function
    def compute_snr_lalsim(m1, m2, z, chi1, chi2, dL_mpc, iota, ra, dec, psi, df, fmin, fmax, f_ref, psd_interp, approximant_str):
        approximant = lalsim.SimInspiralGetApproximantFromString(approximant_str)
        m1=m1 * (1 + z)
        m2=m2 * (1+z)
        m1_kg = float(m1) * lal.MSUN_SI
        m2_kg = float(m2) * lal.MSUN_SI
        distance_m = float(dL_mpc) * 1e6 * lal.PC_SI

    
        # generate FD waveform (LAL FrequencySeries objects)
        hp, hc = lalsim.SimInspiralChooseFDWaveform(m1_kg, m2_kg, float(chi1[0]), float(chi1[1]), float(chi1[2]),
            float(chi2[0]), float(chi2[1]), float(chi2[2]), distance_m, float(iota), 0.0, 0.0, 0.0, 0.0, df, fmin, fmax, f_ref, None, approximant)
    
        if hp is None or hc is None:
            raise RuntimeError("LALSimulation returned None for hp/hc")
    
        # convert LAL FrequencySeries -> numpy arrays (ensure complex dtype)
        hp_arr = np.asarray(hp.data.data)
        hc_arr = np.asarray(hc.data.data)
    
        # build frequency axis that matches the waveform sampling
        n = hp_arr.size
        fs = np.arange(n) * float(df)
    
        # get detector response (Fp, Fc) and time delay dt
        det_obj = lal.cached_detector_by_prefix["H1"]
        gmst = lal.GreenwichMeanSiderealTime(0.0) #replace with time or tgeocenter with lalgps time
        Fp, Fc = lal.ComputeDetAMResponse(det_obj.response,
                                          float(ra), float(dec), float(psi), float(gmst))
        dt = lal.TimeDelayFromEarthCenter(det_obj.location, float(ra), float(dec), 0.0)
    
        # apply antenna pattern and time delay (use numpy arrays)
        phase = np.exp(2j * np.pi * fs * float(dt))
        h_fd = (float(Fp) * hp_arr + float(Fc) * hc_arr) * phase
    
        # PSD sampling and integration mask
        psd_vals = psd_interp(fs)                  # psd_interp should return numpy array
        valid = (fs >= fmin) & (fs <= fmax)
    
        # guard against non-positive PSD values
        psd_valid = np.where(psd_vals[valid] > 0.0, psd_vals[valid], np.inf)
    
        integrand = (np.abs(h_fd[valid])**2) / psd_valid #make sure abs before vs after doesnt make a difference
        snr_sq = 4.0 * np.trapz(integrand, fs[valid])
        return float(np.sqrt(snr_sq))


    # lal phenomD
    snrs_lalsim_D = []
    for _, row in test_df.iterrows():
        chi1 = [row.s1x, row.s1y, row.s1z]
        chi2 = [row.s2x, row.s2y, row.s2z]
        
        snr = compute_snr_lalsim(row.m1, row.m1*row.q, row.z, chi1, chi2, row.dL*1e3, row.iota, row.ra, row.dec,row.psi, 0.25, fmin, fmax, f_ref, psd_interp, "IMRPhenomD")
        snrs_lalsim_D.append(snr)

    # lalsim Pv2
    snrs_lalsim_Pv2 = []
    for _, row in test_df.iterrows():
        chi1 = [row.s1x, row.s1y, row.s1z]
        chi2 = [row.s2x, row.s2y, row.s2z]
        snr = compute_snr_lalsim(row.m1, row.m1*row.q,row.z, chi1, chi2, row.dL*1e3, row.iota, row.ra, row.dec,row.psi,  0.25, fmin, fmax, f_ref, psd_interp, "IMRPhenomPv2")
        snrs_lalsim_Pv2.append(snr)

    
    results = pd.DataFrame({"Event": np.arange(len(test_df)), "SNR ripple phenomD": snrs_jax, "SNR lal phenomD": snrs_lalsim_D,
       # "SNR_Ripple_IMRPhenomPv2": snrs_ripple_Pv2,
                            "SNR lal Pv2": snrs_lalsim_Pv2})

    return results


In [33]:
nevents = 10
z=np.linspace(0.1, 1.8, nevents)

test_df = pd.DataFrame({
    "m1": np.linspace(10, 40, nevents)[-1], #source frame masses
    "q": np.linspace(0.5, 1.0, nevents),
    "z": z,
    "s1x": np.zeros(nevents),
    "s1y": np.zeros(nevents),
    "s1z": np.linspace(0, 1, nevents),
    "s2x": np.zeros(nevents),
    "s2y": np.zeros(nevents),
    "s2z": np.linspace(0, 1, nevents),
    "gps_time": np.zeros(nevents),
    "dL": population_parameters['cosmo'].dL(z), # dL in Gpc ,
    "iota": np.linspace(0, np.pi/2, nevents),
    "psi": np.zeros(nevents),
    "ra": np.linspace(0, 2*np.pi, nevents),
    "dec": np.linspace(-np.pi/4, np.pi/4, nevents) })

# m1, m2, s1x, s1y, s1z, s2x, s2y, s2z, dist_mpc, tc, phiRef, incl = theta

In [34]:
reload(fisher_snrs)

<module 'fisher_snrs' from '/mnt/home/amoran/GW_pop_cosmo_inference/src/fisher_snrs.py'>

In [35]:
SNR_comp=compare_all_snrs(test_df)

/tmp/ipykernel_426064/685247420.py:61: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  snr_sq = 4.0 * np.trapz(integrand, fs[valid])


In [36]:
SNR_comp

,Event,SNR ripple phenomD,SNR lal phenomD,SNR lal Pv2
0,0,23.072448,23.069730,23.069730
1,1,13.542067,13.540568,13.540568
2,2,6.455231,6.454559,6.454559
3,3,9.071163,9.070274,9.070274
4,4,7.316305,7.315625,7.315625
5,5,4.864890,4.864458,4.864458
6,6,1.136815,1.136719,1.136719
7,7,3.136275,3.136020,3.136020
8,8,2.067086,2.066926,2.066926
9,9,3.199619,3.199386,3.199386
